In [1]:
%pylab inline
import networkx as nx
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
import plotly.plotly as py
from plotly.graph_objs import *
import pandas as pd
import numpy as np
import itertools
import plotly.tools as tls
import pickle
import yaml
import os
credentials = yaml.load(open(os.path.expanduser('plotlyCred.yml')))
plotly.tools.set_credentials_file(credentials.get('username'), api_key=credentials.get('api_key'))

Populating the interactive namespace from numpy and matplotlib


IOError: [Errno 2] No such file or directory: 'plotlyCred.yml'

In [ ]:
import sys
reload(sys)
sys.setdefaultencoding('utf-8')

# Loading data

In [2]:
df = spark.read.parquet("s3a://meetupdf/topic_df")

In [3]:
df.show(5)

+--------+---------+-------------+--------------------+----------+
|group_id| event_id|   event_time|               topic|group_city|
+--------+---------+-------------+--------------------+----------+
|18717087|238024816|1494437400000|Communication Skills|    London|
|18628440|237343136|1488299400000|       Microservices| Bucharest|
| 1563505|238023271|1488434400000|         New In Town|     Paris|
| 2849712|238010279|1489201200000|         Sport Bikes|    Corona|
|19091202|238024817|1488477600000|  Startup Businesses|     Paris|
+--------+---------+-------------+--------------------+----------+
only showing top 5 rows



# Convert DF to Pd

In [4]:
def convert_pd_datetime(df):
	'''input: a sparksql dataframe
		output: a panda dataframe that has converted timestamp to datetime '''
	urlkey_df = df.toPandas()
	urlkey_df['datetime'] = pd.to_datetime(urlkey_df['event_time'],unit = 'ms')
	urlkey_df['date'] =  pd.DatetimeIndex(urlkey_df['datetime']).date
	return urlkey_df

In [5]:
urlkey_df = convert_pd_datetime(df)

In [6]:
urlkey_df.head(20)

,group_id,event_id,event_time,topic,group_city,datetime,date
0,18717087,238024816,1.494437e+12,Communication Skills,London,2017-05-10 17:30:00,2017-05-10
1,18628440,237343136,1.488299e+12,Microservices,Bucharest,2017-02-28 16:30:00,2017-02-28
2,1563505,238023271,1.488434e+12,New In Town,Paris,2017-03-02 06:00:00,2017-03-02
3,2849712,238010279,1.489201e+12,Sport Bikes,Corona,2017-03-11 03:00:00,2017-03-11
4,19091202,238024817,1.488478e+12,Startup Businesses,Paris,2017-03-02 18:00:00,2017-03-02
5,12328622,qtbsmmywfbhb,1.488703e+12,Holland,Melbourne,2017-03-05 08:30:00,2017-03-05
6,1135121,tgdbbmywfbcb,1.488415e+12,Running,Chicago,2017-03-02 00:30:00,2017-03-02
7,19538170,qghvrlywpbjc,1.513508e+12,Museum,Gwernaffield,2017-12-17 11:00:00,2017-12-17
8,22048639,237313741,1.488476e+12,JavaScript,Berlin,2017-03-02 17:30:00,2017-03-02
9,22048639,237313741,1.488476e+12,Cloud Computing,Berlin,2017-03-02 17:30:00,2017-03-02


In [7]:
sf_pd = urlkey_df.ix[urlkey_df['group_city'] =="San Francisco"]

In [8]:
def unique_urlkey(df):
    return df['urlkey'].unique()
def top_urlkey_count(df):
    urlkey_count = df.groupby('urlkey')['group_id'].count()
    urlkey_count = pd.DataFrame(urlkey_count)
    return urlkey_count

In [9]:
def timeseries_topn_urlkey(df,n):
    """Input: DataFrame and N: number of urlkeys you want to plot
    Output: DataFrame that have top n values for everyday """
    #get urlkey count
    urlkey_count = df.groupby('topic')['group_id'].count()
    urlkey_count = pd.DataFrame(urlkey_count)
    
    # sort urlkey count in descending order
    ordered_urlkey_count = urlkey_count.sort_values('group_id', ascending=False)
    pop_topics_list = ordered_urlkey_count.index.values[:10].tolist()
    
    # get the top N values
    pop_urlkey_count = ordered_urlkey_count.index.values[:n].tolist()
    
    #create timeseries 
    new_topic_pd = df[['topic','date']]
    topic_index = new_topic_pd['topic'].isin(pop_topics_list)
    top_100_topic_pd = new_topic_pd[topic_index]
    
    #get urlkey count/day 
    crosstab_topic_date = pd.crosstab(top_100_topic_pd['date'],top_100_topic_pd['topic'])
    return crosstab_topic_date

In [10]:
sf_timeseries = timeseries_topn_urlkey(sf_pd,6)

# Plotting data

In [11]:
py.iplot([{
    'x': sf_timeseries.index,
    'y': sf_timeseries[col],
    'name': col
}  for col in sf_timeseries.columns])

In [40]:
# def create_time_series_pd(df):
#     topic_count = df.groupby('topic')['group_id'].count()
#     topic_count = pd.DataFrame(topic_count)
#     ordered_topic_count = topic_count.sort_values('group_id', ascending=False)
# #     pop_topics_list = ordered_topic_count.index.values[:10].tolist()
#     pop_topics_list = ['Tech','Data Science','Web Development','Python', 'Data Analytics','Spark','JavaScript','Hadoop']
#     new_topic_pd = df[['topic','date']]
#     topic_index = df['topic'].isin(pop_topics_list)
#     top_100_topic_pd = new_topic_pd[topic_index]
#     crosstab_topic_date = pd.crosstab(top_100_topic_pd['date'],top_100_topic_pd['topic'])
#     return crosstab_topic_date


In [12]:
def timeseries_specific_topics(df,spec_topics):
#     topic_count = df.groupby('topic')['group_id'].count()
#     topic_count = pd.DataFrame(topic_count)
#     ordered_topic_count = topic_count.sort_values('group_id', ascending=False)
    new_topic_pd = df[['topic','date']]
    topic_index = df['topic'].isin(spec_topics)
    top_100_topic_pd = new_topic_pd[topic_index]
    crosstab_topic_date = pd.crosstab(top_100_topic_pd['date'],top_100_topic_pd['topic'])
    return crosstab_topic_date

In [27]:
datascience = ['JavaScript','Data Science','Python',\
               'Machine Learning','Data Analytics','Hadoop','Galvanize']
datasciencetopics = timeseries_specific_topics(urlkey_df,datascience)

In [28]:
py.iplot([{
    'x': datasciencetopics.index,
    'y': datasciencetopics[col],
    'name': col
}  for col in datasciencetopics.columns])